Resizing the REAL and FAKE images to 256x256

In [34]:
import os
import zipfile
from PIL import Image
import numpy as np
import cv2

real_folder = '/Users/siddharthiyer/Desktop/detect ai artwork - resizing images for efficiency/ai vs real art/REAL'
fake_folder = '/Users/siddharthiyer/Desktop/detect ai artwork - resizing images for efficiency/ai vs real art/FAKE'

def gauss_kernel(size, sigma):
    size = int(size) // 2
    x, y = np.mgrid[-size:size+1, -size:size+1]
    g = np.exp(-((x**2 + y**2) / (2.0*sigma**2))) * (1 / (2.0 * np.pi * sigma**2))
    g /= g.sum() # Normalize
    return g

def subsample(image_array):
    R = image_array[:, :, 0]
    G = image_array[:, :, 1]
    B = image_array[:, :, 2]

    R_sub = R[::2, :]
    G_sub = G[::2, :]
    B_sub = B[::2, :]
    R_sub = R_sub[:, ::2]
    G_sub = G_sub[:, ::2]
    B_sub = B_sub[:, ::2]

    subsampled_array = np.stack([R_sub, G_sub, B_sub], axis=-1)

    return subsampled_array

In [36]:
# Resize images

# Images begin as 512x512. For every image, blur with gauss kernel and convolution code, then subsample to 256x256
# Save the new images in a new folder

blur_kernel = gauss_kernel(5, 1)

real_resized = []
fake_resized = []

for filename in os.listdir(real_folder):
    image = Image.open(real_folder + '/' + filename)
    image = np.array(image)
    image_resized = cv2.filter2D(image, -1, blur_kernel)
    image_resized = subsample(image_resized)
    real_resized.append(image_resized)

for filename in os.listdir(fake_folder):
    image = Image.open(real_folder + '/' + filename)
    image = np.array(image)
    image_resized = cv2.filter2D(image, -1, blur_kernel)
    image_resized = subsample(image_resized)
    fake_resized.append(image_resized)

# Create folders for real and fake resized
# os.mkdir(real_folder + '_resized')
# os.mkdir(fake_folder + '_resized')

# Save images
for i, image in enumerate(real_resized):
    cv2.imwrite(real_folder + '_resized/' + str(i) + '.jpg', image)

for i, image in enumerate(fake_resized):
    cv2.imwrite(fake_folder + '_resized/' + str(i) + '.jpg', image)
